In [3]:
#database access
import psycopg2 as ps
import os
import pprint
#password setter
import getpass

#measuring runtime
from datetime import datetime
import time

#pandas
import pandas as pd
import pylab 
import scipy.stats as stats
#progress bar
from ipywidgets import FloatProgress
from IPython.display import display

#basic utility
import math
import pylab 

#prevent pandas errors
pd.options.mode.chained_assignment = None  # default='warn'

In [4]:
Password = getpass.getpass()

········


In [5]:
category_list = ['Timestamp','Max_bid', 'Max_bid_volume', 'mean_bid_price', 'mean_bid_volume', 'bid_total_volume',\
                     'number_bids', 'Min_ask', 'Min_ask_volume', 'mean_ask_price', 'mean_ask_volume',\
                     'ask_total_volume', 'number_asks', 'VWAP_20_bid', 'VWAP_20_ask', 'VWAP_10_bid', 'VWAP_10_ask',\
                     'av_bid_volume_price_ratio', 'av_ask_volume_price_ratio', 'twenty_percent_volume_b',\
                     'ten_percent_volume_b', 'five_percent_volume_b', 'twenty_percent_volume_a', \
           'ten_percent_volume_a', 'five_percent_volume_a', 'top_twenty_volume_b', 'top_ten_volume_b', 'top_five_volume_b', \
           'top_twenty_volume_a', 'top_ten_volume_a', 'top_five_volume_a', 'twenty_last_added_volume_b', \
           'ten_last_added_volume_b', 'five_last_added_volume_b', 'twenty_last_added_volume_a', \
           'ten_last_added_volume_a', 'five_last_added_volume_a', 'twenty_percent_mid_dev_volume_b', \
           'ten_percent_mid_dev_volume_b', 'five_percent_mid_dev_volume_b', 'twenty_percent_mid_dev_volume_a', \
           'ten_percent_mid_dev_volume_a', 'five_percent_mid_dev_volume_a']

statistics_list = ['Mid', 'spread', 'VWAP_20_n_bid', 'VWAP_20_n_ask', 'VWAP_20_n_spread', 'VWAP_10_n_bid', \
              'VWAP_10_n_ask', 'VWAP_10_n_spread', 'log_returns', 'Vola_last_10']
category_dict = {}
for i in category_list:
    category_dict[i]=[]


In [6]:
def reconstruct_orderbook(currency_pair, exchange, Database):
    
    #takes a currency_pair and exchange and returns a pandas dataframe containing all the orderbooks

    conn = ps.connect("dbname = '{0}' user = 'postgres' host='localhost' password = '{1}'".format(Database, Password))
    cur = conn.cursor()
    cur.execute("SELECT * FROM orderbook WHERE orderbook.currency_pair_id='{0}' and \
        orderbook.exchange_id='{1}'".format(currency_pair,exchange) )
    
    orderbook = cur.fetchall()
    temp = ['orderbook_id', 'exchange_id', 'currency_pair_id', 'exchange_timestamp'
        , 'saved_timestamp', 'parent_orderbook_id', 'type']

    Data = {'orderbook_id':[], 'exchange_id':[], 'currency_pair_id':[], 'exchange_timestamp':[] \
            , 'saved_timestamp':[], 'parent_orderbook_id':[], 'type':[]}
    
    for i in range(0,len(orderbook)):
        for j in range(0,len(Data)):
            Data[temp[j]].append(orderbook[i][j])
        
    df = pd.DataFrame(data = Data)
    conn.close()
    
    orderbook_dataframe = df.sort_values(by = 'orderbook_id')
    
    
    return(orderbook_dataframe)

def sort_D_orderbooks(df):
    #sorts Delta orderbooks into two dataframes containing the Deletes and the New orders and returns those dfs
    df_temp_Del = df.loc[df['status'] == 'DEL']
    df_temp_New = df.loc[df['status'] == 'PNE']
    return(df_temp_Del, df_temp_New)

def reconstruct_orderbook_snapshots(Id, Database):
    #fetches the orderbook given by the Id
    conn = ps.connect("dbname = '{0}' user = 'postgres' host='localhost' password = '{1}'".format(Database, Password))
    cur = conn.cursor()
    cur.execute("SELECT * FROM limit_order WHERE limit_order.orderbook_id='{0}'".format(Id) )
    
    orderbook = cur.fetchall()
    temp = ['orderbook_id', 'order_id', 'limit_price', 'original_amount', 'status', 'order_type']
    limit_order_df = pd.DataFrame(data = orderbook, columns = temp)#columns dazu, pandas funktionsweise
    
    return(limit_order_df)



In [7]:
def get_VWAP(orderbook, amount, order_type):
    df = orderbook.loc[orderbook['order_type'] == order_type]
    
    if order_type == 'B':
        df.sort_values(by = ['limit_price'], ascending = False, inplace = True)
    elif order_type == 'A':
        df.sort_values(by = ['limit_price'], ascending = True, inplace = True)
    
    if df.empty == True:
        return(None)
    if df['original_amount'].head(1).iloc[0] >= amount:
        VWAP =  df['limit_price'].head(1).iloc[0]
        return(VWAP)
    df['temp'] = df['original_amount'].cumsum()
    df['costs'] = df['original_amount'] * df['limit_price']
    df['cum_costs'] = df['costs'].cumsum()
    last_full_order_index = df.loc[df['temp'] < amount].tail(1).index[0]
    last_price = df.shift(-1).loc[last_full_order_index]['limit_price']
    if type(last_price) == float:
        VWAP = None
    else:
        VWAP = (last_price*(amount - df['temp'][last_full_order_index]) + df['cum_costs'][last_full_order_index])/amount
    return(VWAP)

def order_size_to_price_ratio(orderbook, Max_bid, Min_ask):
    Mid = (Max_bid+Min_ask)/2
    ob_a = orderbook.loc[orderbook['order_type'] == 'A']
    ob_b = orderbook.loc[orderbook['order_type'] == 'B']
    ob_a['size_to_price'] = (ob_a['limit_price']-Mid)/ob_a['original_amount']
    ob_b['size_to_price'] = (Mid-ob_b['limit_price'])/ob_b['original_amount']

    avg_B = ob_b['size_to_price'].mean()
    avg_A = ob_a['size_to_price'].mean()
    
    return([avg_B, avg_A])

def get_volume_imbalances(orderbook, amount, restriction):
    
    #restricted percentage
    df_1 = restrict_orderbook(orderbook, amount=amount, how=restriction)
    df_1_b = df_1.loc[df_1['order_type'] == 'B']
    df_1_a = df_1.loc[df_1['order_type'] == 'A']
    bid = df_1_b['original_amount'].sum()
    ask = df_1_a['original_amount'].sum()
    
    return([bid, ask])

def get_statistics(orderbook):
    #helper that actually gets all the requested Data from a orderbook
    if 'VWAP_20_bid' in category_dict:
        VWAP_20_bid = get_VWAP(orderbook, 20, 'B')
        category_dict['VWAP_20_bid'].append(VWAP_20_bid)
    
    if 'VWAP_20_ask' in category_dict:
        VWAP_20_ask = get_VWAP(orderbook, 20, 'A')
        category_dict['VWAP_20_ask'].append(VWAP_20_ask)
        
    if 'VWAP_10_bid' in category_dict:
        VWAP_10_bid = get_VWAP(orderbook, 10, 'B')
        category_dict['VWAP_10_bid'].append(VWAP_10_bid)
        
    if 'VWAP_10_ask' in category_dict:
        VWAP_10_ask = get_VWAP(orderbook, 10, 'A')
        category_dict['VWAP_10_ask'].append(VWAP_10_ask)
        
        
    orderbook_bid = orderbook.loc[orderbook['order_type']=='B']
    if 'Max_bid' in category_dict:
        Max_bid = orderbook_bid['limit_price'].max()
        category_dict['Max_bid'].append(Max_bid)
        
    if 'Max_bid_volume' in category_dict:
        Max_bid_volume = orderbook['original_amount'].loc[orderbook_bid['limit_price'].astype(float).idxmax()]
        category_dict['Max_bid_volume'].append(Max_bid_volume)
        
    if 'bid_total_volume' in category_dict:
        bid_total_volume = orderbook_bid['original_amount'].sum()
        category_dict['bid_total_volume'].append(bid_total_volume)
        
    if 'number_bids' in category_dict:
        number_bids = orderbook_bid.shape[0]
        category_dict['number_bids'].append(number_bids)
        
    if 'mean_bid_price' in category_dict:
        mean_bid_price = orderbook_bid['limit_price'].mean()
        category_dict['mean_bid_price'].append(mean_bid_price)
        
    if 'mean_bid_volume' in category_dict:
        mean_bid_volume = orderbook_bid['original_amount'].mean()
        category_dict['mean_bid_volume'].append(mean_bid_volume)
    
    
    orderbook_ask = orderbook.loc[orderbook['order_type']=='A']
    if 'Min_ask' in category_dict:
        Min_ask = orderbook_ask['limit_price'].min()
        category_dict['Min_ask'].append(Min_ask)
        
    if 'Min_ask_volume' in category_dict:
        Min_ask_volume = orderbook_ask['original_amount'].loc[orderbook_ask['limit_price'].astype(float).idxmin()]
        category_dict['Min_ask_volume'].append(Min_ask_volume)
        
    if 'ask_total_volume' in category_dict:
        ask_total_volume = orderbook_ask['original_amount'].sum()
        category_dict['ask_total_volume'].append(ask_total_volume)
        
    if 'number_asks' in category_dict:
        number_asks = orderbook_ask.shape[0]
        category_dict['number_asks'].append(number_asks)
        
    if 'mean_ask_price' in category_dict:
        mean_ask_price = orderbook_ask['limit_price'].mean()
        category_dict['mean_ask_price'].append(mean_ask_price)
        
    if 'mean_ask_volume' in category_dict:
        mean_ask_volume = orderbook_ask['original_amount'].mean()
        category_dict['mean_ask_volume'].append(mean_ask_volume)

    if 'av_bid_volume_price_ratio' in category_dict:
        temp = order_size_to_price_ratio(orderbook, Max_bid, Min_ask)
        av_bid_volume_price_ratio = temp[0]
        av_ask_volume_price_ratio = temp[1]
        category_dict['av_bid_volume_price_ratio'].append(av_bid_volume_price_ratio)
        category_dict['av_ask_volume_price_ratio'].append(av_ask_volume_price_ratio)

    #volume imbalances
    #block1
    if 'top_twenty_volume_b' in category_dict or 'top_twenty_volume_a' in category_dict:
        twenty_percent_volume = get_volume_imbalances(orderbook, amount=20, restriction=2)
        category_dict['top_twenty_volume_b'].append(twenty_percent_volume[0])
        category_dict['top_twenty_volume_a'].append(twenty_percent_volume[1])

    if 'top_ten_volume_b' in category_dict or 'top_ten_volume_a' in category_dict:
        ten_percent_volume = get_volume_imbalances(orderbook, amount=10, restriction=2)
        category_dict['top_ten_volume_b'].append(ten_percent_volume[0])   
        category_dict['top_ten_volume_a'].append(ten_percent_volume[1])      

    if 'top_five_volume_b' in category_dict or 'top_five_volume_a' in category_dict:
        five_percent_volume = get_volume_imbalances(orderbook, amount=5, restriction=2)
        category_dict['top_five_volume_b'].append(twenty_percent_volume[0])
        category_dict['top_five_volume_a'].append(twenty_percent_volume[1])

    #block2
    if 'twenty_percent_volume_b' in category_dict or 'twenty_percent_volume_a' in category_dict:
        twenty_percent_volume = get_volume_imbalances(orderbook, amount=20, restriction=3)
        category_dict['twenty_percent_volume_b'].append(twenty_percent_volume[0])
        category_dict['twenty_percent_volume_a'].append(twenty_percent_volume[1])

    if 'ten_percent_volume_b' in category_dict or 'ten_percent_volume_a' in category_dict:
        ten_percent_volume = get_volume_imbalances(orderbook, amount=10, restriction=3)
        category_dict['ten_percent_volume_b'].append(ten_percent_volume[0])
        category_dict['ten_percent_volume_a'].append(ten_percent_volume[1])

    if 'five_percent_volume_b' in category_dict or 'five_percent_volume_a' in category_dict:
        five_percent_volume = get_volume_imbalances(orderbook, amount=5, restriction=3)
        category_dict['five_percent_volume_b'].append(five_percent_volume[0])
        category_dict['five_percent_volume_a'].append(five_percent_volume[1])
        
    #block3
    if 'twenty_percent_mid_dev_volume_b' in category_dict or 'twenty_percent_mid_dev_volume_a' in category_dict:
        #return('Hier')
        twenty_percent_mid_dev_volume = get_volume_imbalances(orderbook, amount=20, restriction=4)
        category_dict['twenty_percent_mid_dev_volume_b'].append(twenty_percent_mid_dev_volume[0])
        category_dict['twenty_percent_mid_dev_volume_a'].append(twenty_percent_mid_dev_volume[1])
    
    if 'ten_percent_mid_dev_volume_b' in category_dict or 'ten_percent_mid_dev_volume_a' in category_dict:
        ten_percent_mid_dev_volume = get_volume_imbalances(orderbook, amount=10, restriction=4)
        category_dict['ten_percent_mid_dev_volume_b'].append(ten_percent_mid_dev_volume[0])
        category_dict['ten_percent_mid_dev_volume_a'].append(ten_percent_mid_dev_volume[1])

    if 'five_percent_mid_dev_volume_a' in category_dict or 'five_percent_mid_dev_volume_a' in category_dict:
        five_percent_mid_dev_volume = get_volume_imbalances(orderbook, amount=5, restriction=4)
        category_dict['five_percent_mid_dev_volume_b'].append(five_percent_mid_dev_volume[0])
        category_dict['five_percent_mid_dev_volume_a'].append(five_percent_mid_dev_volume[1])
    
    #block4
    if 'twenty_last_added_volume_b' in category_dict or 'twenty_last_added_volume_a' in category_dict:
        twenty_last_added_volume = get_volume_imbalances(orderbook, amount=20, restriction=5)
        category_dict['twenty_last_added_volume_b'].append(twenty_last_added_volume[0])
        category_dict['twenty_last_added_volume_a'].append(twenty_last_added_volume[1])
    
    if 'ten_last_added_volume_b' in category_dict or 'ten_last_added_volume_a' in category_dict:
        ten_last_added_volume = get_volume_imbalances(orderbook, amount=10, restriction=5)
        category_dict['ten_last_added_volume_b'].append(ten_last_added_volume[0])
        category_dict['ten_last_added_volume_a'].append(ten_last_added_volume[1])
        
    if 'five_last_added_volume_b' in category_dict or 'five_last_added_volume_a' in category_dict:
        five_last_added_volume = get_volume_imbalances(orderbook, amount=5, restriction=5)
        category_dict['five_last_added_volume_b'].append(five_last_added_volume[0])
        category_dict['five_last_added_volume_a'].append(five_last_added_volume[1])

    results = []
    for i in category_list[1:len(category_list)]:
        results.append(category_dict[i][-1])
    
    return( results )


In [29]:
def summarize_order_books_gamma(currency_pair, exchange, n, Database):

    
    df_orderbook = reconstruct_orderbook(currency_pair, exchange, Database)
    
    if df_orderbook.empty:
        
        results = pd.DataFrame(columns = category_list + statistics_list)
        return(results)
    
    df_orderbook_F = df_orderbook.loc[df_orderbook['type']=='F']
    df_orderbook_F['next_index'] = df_orderbook_F.shift(1)['orderbook_id']
    df_orderbook_F['Id_tuple'] = df_orderbook_F[['orderbook_id', 'next_index']].apply(tuple, axis=1)
    df_orderbook_F['Id_list'] = df_orderbook_F['Id_tuple'].apply(( \
        lambda x: select_deltas_between_F_Ids(df_orderbook, x)))
    
    df_orderbook_F['Id_list'] = df_orderbook_F['Id_list'].shift(-1)
    df_orderbook_F['Timestamp_list'] = df_orderbook_F['Id_tuple'].apply(( \
        lambda x: build_timestamp_list(df_orderbook, x)))
    df_orderbook_F['Timestamp_list'] = df_orderbook_F['Timestamp_list'].shift(-1)
    
    
    if n is None:
        n = df_orderbook_F['orderbook_id'].size

    elif n > df_orderbook_F['orderbook_id'].size:
        n = df_orderbook_F['orderbook_id'].size

    f = FloatProgress(min =0,  max = n)
    display(f)
    runs = 0
    for Id in df_orderbook_F['orderbook_id'][0:n]:
        df_loop  = reconstruct_orderbook_snapshots(Id, Database)
        runs += 1
        if type(df_orderbook_F.loc[df_orderbook_F['orderbook_id'] == Id]['Id_list'].tolist()[0]) == list:
            if len(df_orderbook_F.loc[df_orderbook_F['orderbook_id'] == Id]['Id_list'].tolist()[0]) != 0:
                new_stats = go_multiple_steps(df_loop, df_orderbook_F.loc[df_orderbook_F['orderbook_id'] == Id]\
                            ['Id_list'], df_orderbook_F.loc[df_orderbook_F['orderbook_id'] == Id]['Timestamp_list'],\
                                               Database)
            else:
                loop_data_1 = get_statistics(df_loop)
                loop_data_2 = [df_orderbook_F.loc[df_orderbook_F['orderbook_id']==Id]['exchange_timestamp']] + loop_data_1
                new_stats = pd.DataFrame(columns = category_list)
                new_stats.loc[0] = loop_data_2
        else:
            loop_data_1 = get_statistics(df_loop)
            loop_data_2 = [df_orderbook_F.loc[df_orderbook_F['orderbook_id']==Id]['exchange_timestamp']] + loop_data_1
            new_stats = pd.DataFrame(columns = category_list)
            new_stats.loc[0] = loop_data_2
        
        if runs == 1:
            df_stats = new_stats
        else:
            df_stats = pd.concat([df_stats, new_stats])
        f.value +=1
        
    if 'Mid' in statistics_list:
        df_stats['Mid'] = (df_stats['Min_ask'] + df_stats['Max_bid']) /2 
    if 'spread' in statistics_list:
        df_stats['spread'] = (df_stats['Min_ask'] - df_stats['Max_bid'])/df_stats['Mid']
    
    if 'VWAP_20_n_bid' in statistics_list:
        df_stats['VWAP_20_n_bid'] = (df_stats['VWAP_20_bid'] - df_stats['Mid'])/df_stats['Mid']
    if 'VWAP_20_n_ask' in statistics_list:
        df_stats['VWAP_20_n_ask'] = (df_stats['VWAP_20_ask'] - df_stats['Mid'])/df_stats['Mid']
    if 'VWAP_20_n_spread' in statistics_list:
        df_stats['VWAP_20_n_spread'] = df_stats['VWAP_20_n_ask'] - df_stats['VWAP_20_n_bid']
    
    if 'VWAP_10_n_bid' in statistics_list:
        df_stats['VWAP_10_n_bid'] = (df_stats['VWAP_10_bid'] - df_stats['Mid'])/df_stats['Mid']
    if 'VWAP_10_n_ask' in statistics_list:
        df_stats['VWAP_10_n_ask'] = (df_stats['VWAP_10_ask'] - df_stats['Mid'])/df_stats['Mid']
    if 'VWAP_10_n_spread' in statistics_list:
        df_stats['VWAP_10_n_spread'] = df_stats['VWAP_10_n_ask'] - df_stats['VWAP_10_n_bid']
    
    if 'log_returns' in statistics_list:
        df_stats['log_returns'] = df_stats.shift(1)['Mid']/df_stats['Mid']
        df_stats['log_returns'] = df_stats['log_returns'].astype(float).apply(math.log)
    if 'Vola_last_10' in statistics_list:
        df_stats['Vola_last_10'] = df_stats['log_returns'].rolling(10).std().shift(-10)
        #muss quadriert sein oder std


    df_stats.set_index('Timestamp', inplace = True)
    
    return(df_stats)

def select_deltas_between_F_Ids(df, ids):
    #returns the deltas between two Full orderbooks with ids id1 and id2 where id1 is the orderbook that occured first
    id1 = ids[1]
    id2 = ids[0]
    df = df.loc[(df['orderbook_id']>id1) & (df['orderbook_id']<id2)]
    Id_list = df['orderbook_id'].values.tolist()
    return(Id_list)

def build_timestamp_list(df, ids):
    #returns the deltas between two Full orderbooks with ids id1 and id2 where id1 is the orderbook that occured first
    id1 = ids[1]
    id2 = ids[0]
    df = df.loc[(df['orderbook_id']>id1) & (df['orderbook_id']<id2)]
    timestamps = df['exchange_timestamp'].tolist()
    return(timestamps)


def go_multiple_steps(df_loop, Id_chunks, timestamps, Database):
    
    conn = ps.connect("dbname = '{0}' user = 'postgres' host='localhost' password = '{1}'".format(Database, Password))
    cur = conn.cursor()
    Id_chunks = Id_chunks.iloc[0]
    for_sql = tuple(Id_chunks)
    if len(Id_chunks) == 1:
        for_sql = '('+str(Id_chunks[0])+')'
    
    cur.execute("SELECT * FROM limit_order WHERE limit_order.orderbook_id IN {0}".format(for_sql))
    
    orderbook = cur.fetchall()
    temp = ['orderbook_id', 'order_id', 'limit_price', 'original_amount', 'status', 'order_type']
        
    limit_order_df = pd.DataFrame(data = orderbook, columns = temp)
    df_stats = pd.DataFrame(columns = category_list)
    for count, Id in enumerate(Id_chunks):
        df_temp       = limit_order_df.loc[limit_order_df['orderbook_id']== Id]
        df_loop       = Apply_delta(df_loop, df_temp)
        loop_data = get_statistics(df_loop)
        df_stats_temp = pd.DataFrame(columns = category_list[1:len(category_list)])
        df_stats_temp.loc[0] = loop_data
        df_stats      = pd.concat([df_stats, df_stats_temp])
    df_stats.drop(columns = ['Timestamp'], inplace=True)
    df_stats.insert(loc = 0, column = 'Timestamp', value = timestamps.tolist()[0])
    return(df_stats)

def Apply_delta(df,df2):
    # takes two orderbook, df2 being a Delta, and applying the changes in df2 to df, returning the new frame
    df_Last = df
    df_Delta = df2
    
    if df_Last.index.name != 'order_id':
        df_Last.set_index('order_id', inplace = True)
    
    df_del, df_new = sort_D_orderbooks(df_Delta)
    df_new.set_index('order_id', inplace = True)
    df_del['order_id'] = df_del['order_id'] * (-1)
    
    #add new Transactions
    df_Last = pd.concat([df_Last, df_new])
    
    #delete old Transactions
    drop = df_del['order_id'].values.tolist()
    df_Last.drop(labels = drop, inplace = True)
    df_Last.reset_index(inplace = True)
    return (df_Last)



In [11]:
def restrict_orderbook(orderbook, how, amount):
    #takes a orderbook without deletes and filters it after a condition of choice
    df_B = orderbook.loc[orderbook['order_type'] == 'B']
    df_A = orderbook.loc[orderbook['order_type'] == 'A']
    
    mid_price = (df_B['limit_price'].max() + df_A['limit_price'].min() )/2
    
    if how == 0:
        #restrict to amount Units -not working
        df_B.sort_values(by = ['limit_price'], ascending = False, inplace = True)
        df_A.sort_values(by = ['limit_price'], ascending = True, inplace = True)
        df_B['cumsum'] = df_B['original_amount'].cumsum()
        df_A['cumsum'] = df_A['original_amount'].cumsum()
        df_B = df_B.loc[df_B['cumsum'] <= amount]
        df_A = df_A.loc[df_A['cumsum'] <= amount]
        df = pd.concat([df_B, df_A])
    
    if how == 1:
        #restrict to amount Units -not working
        df_B.sort_values(by = ['limit_price'], ascending = False, inplace = True)
        df_A.sort_values(by = ['limit_price'], ascending = True, inplace = True)
        df_B['cumsum'] = df_B['original_amount'].cumsum()
        df_A['cumsum'] = df_A['original_amount'].cumsum()
        last_B = df_B.loc[df_B['cumsum'] > amount].head(1).index[0]
        ult_B = df_B.loc[last_B]
        last_A = df_A.loc[df_A['cumsum'] > amount].head(1).index[0]
        ult_A = df_A.loc[last_A]
        df_B = df_B.loc[df_B['cumsum'] <= amount]
        df_A = df_A.loc[df_A['cumsum'] <= amount]
        df_B.append(ult_B)
        df_A.append(ult_A)
        df = pd.concat([df_B, df_A])
        
        
    if how == 2:
        #restrict to top amount entries
        df_B.sort_values(by = ['limit_price'], ascending = False, inplace = True)
        df_A.sort_values(by = ['limit_price'], ascending = True, inplace= True)
        df_B = df_B[0:amount]
        df_A = df_A[0:amount]
        df = pd.concat([df_B, df_A])
    
    if how == 3:
        #restrict to top amount percent
        df_B.sort_values(by = ['limit_price'], ascending = False, inplace = True)
        df_A.sort_values(by = ['limit_price'], ascending = True, inplace= True)
        perc_B = int(df_B.shape[0]*amount/100)
        perc_A = int(df_A.shape[0]*amount/100)
        df_B = df_B[0:perc_B]
        df_A = df_A[0:perc_A]
        df = pd.concat([df_B, df_A])
    
    if how == 4:
        #percentege of deviation form the mid price
        mid = ( df_B['limit_price'].max() + df_A['limit_price'].min() )/2
        df_B = df_B.loc[df_B['limit_price'] >= int( mid* (100-amount)/100 ) ]
        df_A = df_A.loc[df_A['limit_price'] <= int( mid* (100+amount)/100 ) ]
        df = pd.concat([df_B, df_A])


    if how == 5:
        #last n entries that were added to the book
        if orderbook.index.name == 'order_id':
            df_B.reset_index(inplace = True)
            df_A.reset_index(inplace = True)
        df_B.sort_values(by = ['order_id'], ascending = True, inplace = True)
        df_A.sort_values(by = ['order_id'], ascending = True, inplace = True)
        df_B = df_B.tail(amount)
        df_A = df_A.tail(amount)
        df = pd.concat([df_B, df_A])
        if orderbook.index.name == 'order_id':
            df.set_index('order_id')

    return(df)


In [12]:
def pandas_time_object_to_seconds(pt):
    return(pt.to_pydatetime().time().second)
#check if 'Timestamp'.dt.time() works 
#check with december 2017
def filter_to_minutes(df):
    check = 0
    temp = df
    if temp.index.name == 'Timestamp':
        check = 1
        temp.reset_index(inplace = True)
    temp['temp'] = temp['Timestamp']
    temp['temp'] = temp['temp'].apply(pandas_time_object_to_seconds) 
    temp = temp.loc[(temp['temp'] <= 10) | (temp['temp'] >= 50)]
    temp.drop(['temp'], axis = 1, inplace = True)
    if check ==1:
        temp.set_index('Timestamp', inplace = True)
    return(temp)

In [13]:
def get_volume_imbalances(orderbook, amount, restriction):
    
    #restricted percentage
    df_1 = restrict_orderbook(orderbook, amount=amount, how=restriction)
    df_1_b = df_1.loc[df_1['order_type'] == 'B']
    df_1_a = df_1.loc[df_1['order_type'] == 'A']
    bid = df_1_b['original_amount'].sum()
    ask = df_1_a['original_amount'].sum()
    
    return([bid, ask])


In [14]:
def get_exchange_and_currency_number_from_db(Database, Currency_pair, exchange):
    conn = ps.connect("dbname = '{0}' user = 'postgres' host='localhost' password = '{1}'".format(Database, Password))
    cur = conn.cursor()
    cur.execute("SELECT currency_pair_id FROM currency_pair WHERE currency_pair.name='{0}'".format(Currency_pair) )
    currency_id = cur.fetchall()[0][0]
    
    cur.execute("SELECT exchange_id FROM exchange WHERE exchange.name ='{0}'".format(exchange))
    temp = cur.fetchall()
    exchange_id = []
    for i in range(len(temp)):
        exchange_id.append(temp[i][0])
    return(currency_id, exchange_id)

In [15]:
def get_full_information(currency_pair, exchange):
    Database_list = [
        'Cryptoccy', 
        'Cryptoccy2', 
        'Cryptoccy3', 
        'Cryptoccy4', 
        'Cryptoccy5', 
        'Cryptodb_btc1', 
        #'Cryptodb_btc2'
    ]

    Data = {}
    for i in category_list:
        Data[i] = []
    
    full_category_list = category_list + statistics_list
    results = pd.DataFrame(columns = full_category_list)
    f = FloatProgress(min =0,  max = len(Database_list))
    display(f)
    counter = 0
    for db in Database_list:
        counter += 1
        print(counter)
        temp = get_exchange_and_currency_number_from_db(db, currency_pair, exchange)
        if type(temp[0]) == int and type(temp[1]) == list or type(temp[1]) == int:
            
            if type(temp[1]) == list:
                if len(temp[1]) == 0:
                    df_temp = pd.DataFrame(columns = full_category_list)
                else:
                    for i in temp[1]:
                        df_temp = summarize_order_books_gamma(temp[0],i, 1, db)
                        a.append(results.shape[1])
                        b.append(df_temp.shape[1])
                        results = pd.concat([results, df_temp])
            else:
                df_temp = summarize_order_books_gamma(temp[0], temp[1], 1, db)
            
            if df_temp.empty:
                pass
            else:
                a.append(results.shape[1])
                b.append(df_temp.shape[1])
                results = pd.concat([results, df_temp])
            f.value += 1
        else:
            pass
    results.drop(columns=['Timestamp'], inplace = True)
    results.index.name = 'Timestamp'
    filter_to_minutes(df=results)
    return(results)

In [16]:
testframe= summarize_order_books_gamma(1,1,7, 'Cryptodb_btc2')

FloatProgress(value=0.0, max=7.0)

C:\Users\janpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:127: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [18]:
testframe['Max_bid_shift'] = testframe['Max_bid'].shift(1)

In [20]:
testframe['Max_bid_shift']

Timestamp
2017-12-02 10:33:56+01:00                 NaN
2017-12-02 10:34:08+01:00    11086.0000000000
2017-12-02 10:34:18+01:00    11086.0000000000
2017-12-02 10:34:31+01:00    11086.0000000000
2017-12-02 10:34:37+01:00    11085.0000000000
2017-12-02 10:34:44+01:00    11080.0000000000
2017-12-02 10:34:59+01:00    11076.0000000000
2017-12-02 10:35:05+01:00    11070.0000000000
2017-12-02 10:35:18+01:00    11070.0000000000
2017-12-02 10:35:28+01:00    11066.0000000000
2017-12-02 10:35:35+01:00    11060.0000000000
2017-12-02 10:35:49+01:00    11063.0000000000
2017-12-02 10:35:58+01:00    11061.0000000000
2017-12-02 10:36:08+01:00    11064.0000000000
2017-12-02 10:36:15+01:00    11068.0000000000
2017-12-02 10:36:28+01:00    11069.0000000000
2017-12-02 10:36:38+01:00    11069.0000000000
2017-12-02 10:36:48+01:00    11069.0000000000
2017-12-02 10:36:57+01:00    11069.0000000000
2017-12-02 10:37:07+01:00    11069.0000000000
2017-12-02 10:37:19+01:00    11069.0000000000
2017-12-02 10:37:29+01:0

In [ ]:
start = time.time()
test = get_full_information(currency_pair='BTC/USD', exchange='BitFinex')
end = time.time()
print(end-start)

FloatProgress(value=0.0, max=6.0)

1


FloatProgress(value=0.0, max=491.0)

C:\Users\janpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:126: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\janpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [113]:
start = time.time()
BtcUsdDf = get_full_information(currency_pair='BTC/USD', exchange='BitFinex')
end = time.time()
print(end-start)

FloatProgress(value=0.0, max=6.0)

1


FloatProgress(value=0.0, max=491.0)

C:\Users\janpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:126: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\janpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\janpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



FloatProgress(value=0.0, max=5.0)

...

FloatProgress(value=0.0, max=137.0)

FloatProgress(value=0.0, max=76.0)

6


FloatProgress(value=0.0, max=57.0)

109775.13666796684


In [115]:
BtcUsdDf

,Max_bid,Max_bid_volume,Mid,Min_ask,Min_ask_volume,VWAP_10_ask,VWAP_10_bid,VWAP_10_n_ask,VWAP_10_n_bid,VWAP_10_n_spread,...,top_ten_volume_a,top_ten_volume_b,top_twenty_volume_a,top_twenty_volume_b,twenty_last_added_volume_a,twenty_last_added_volume_b,twenty_percent_mid_dev_volume_a,twenty_percent_mid_dev_volume_b,twenty_percent_volume_a,twenty_percent_volume_b
Timestamp,,,,,,,,,,,,,,,,,,,,,
2018-05-14 17:06:02+02:00,8760.3000000000,0.5800000000,8760.3500000000,8760.4000000000,3.7914189600,8762.23304536400000000000,8757.67088775040000000000,0.0002149509282163383883064032830,-0.0003058225127534858767058393786,4.56215761360000000000,...,69.4316260800,8.2117046900,91.5150041400,16.3174559000,81.0091050000,246.6140166800,7237.6651078200,9969.2923422800,3448.3183889900,4356.8934702000
2018-05-14 17:06:58+02:00,8754.4000000000,1.3700000000,8754.6000000000,8754.8000000000,0.7651309200,8757.20189001520000000000,8752.04473500000000000000,0.0002972026152194275009709181459,-0.0002918768418888355835789185114,5.15715501520000000000,...,12.6740891500,48.9438319300,83.7412039000,57.1446833300,637.0731557400,101.6172576300,7216.9322634800,10013.1394625800,3424.2428763200,4378.9161148400
2018-05-14 17:08:03+02:00,8757.0000000000,0.7029999900,8758.4000000000,8759.8000000000,0.0447918300,8762.64975841620000000000,8751.79136099300000000000,0.0004852208641075995615637559372,-0.0007545486626552795031055900621,10.85839742320000000000,...,64.8874345700,9.7546999900,95.3281659900,54.5887679400,595.6373347300,460.7669726000,7221.2750093500,9946.1072778200,3436.7315513800,4318.1902371200
2018-05-14 17:09:02+02:00,8754.9000000000,0.5959954500,8755.2000000000,8755.5000000000,0.0593951200,8762.65729843380000000000,8751.62900397510000000000,0.0008517564914336622807017543860,-0.0004078714392475328947368421053,11.02829445870000000000,...,8.1277089800,48.3110430600,81.6222114200,65.9182847500,101.1349880000,766.6669209800,7230.6142185200,9941.3293134100,3444.7420615100,4319.7841679200
2018-05-14 17:09:59+02:00,8750.0000000000,29.1650100000,8750.0500000000,8750.1000000000,4.9931720400,8750.60028279600000000000,8750.0000000000,0.00006288910303369695030314112491,-0.000005714253061411077650984851515,0.60028279600000000000,...,22.0996171000,39.4155565000,112.8870088400,50.8038565000,415.3395870700,18448.1821402700,7037.8469015300,9911.7651465200,3253.2609329700,4085.3537066700
2018-05-14 17:10:58+02:00,8736.7000000000,2.1600000000,8736.7500000000,8736.8000000000,5.9933215800,8737.40090176300000000000,8734.29388065390000000000,0.00007450158960711935216184507969,-0.0002811250574984977251266203108,3.10702110910000000000,...,29.8758359700,10.4360297300,49.2581221800,21.8915768700,316.0159098400,316.9462806100,6812.9836378500,9928.4246261300,3174.6897901800,4097.2930289100
2018-05-14 17:12:03+02:00,8742.5000000000,1.1928000000,8742.5500000000,8742.6000000000,0.2696795100,8750.18280508170000000000,8742.29941600000000000000,0.0008730639323423943815019645298,-0.00002866257556433763604440352071,7.88338908170000000000,...,5.0751011800,17.5203081200,15.5252414100,32.9964529500,120.5434093300,179.3053218200,6810.1756734000,9934.6558247100,3170.8995482900,4101.4914449100
2018-05-14 17:13:02+02:00,8743.5000000000,14.2224673600,8743.5500000000,8743.6000000000,14.2828022200,8743.6000000000,8743.5000000000,0.000005718501066500448902333720285,-0.000005718501066500448902333720285,0.1000000000,...,32.7870579600,24.2804673600,49.8352937900,28.3037890900,93.1035596600,35.5327137800,6828.6863184700,9901.0452511400,3268.4734705500,4062.8448269400
2018-05-14 17:14:01+02:00,8749.4000000000,0.0804220800,8749.4500000000,8749.5000000000,0.1255746300,8755.01837062010000000000,8748.98563847640000000000,0.0006364252175965346393201858403,-0.00005307322444267925412454497140,6.03273214370000000000,...,5.2924658500,22.0860484500,93.2531400100,34.1049263900,113.8971529600,516.1630200800,6788.8106737900,9934.5324373200,3227.6695606500,4096.5241028700


In [114]:
BtcUsdDf.to_hdf('C:/Users/janpa/Desktop/Uni/Bachelorarbeit/gathered_stats/BTC_USD_BitFinex_full.h5', \
               key = 'BitFinex', mode = 'w')

C:\Users\janpa\Anaconda3\lib\site-packages\pandas\io\pytables.py:279: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->axis1] [items->None]

  f(store)
C:\Users\janpa\Anaconda3\lib\site-packages\pandas\core\generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->decimal,key->block0_values] [items->['Max_bid', 'Max_bid_volume', 'Mid', 'Min_ask', 'Min_ask_volume', 'VWAP_10_ask', 'VWAP_10_bid', 'VWAP_10_n_ask', 'VWAP_10_n_bid', 'VWAP_10_n_spread', 'VWAP_20_ask', 'VWAP_20_bid', 'VWAP_20_n_ask', 'VWAP_20_n_bid', 'VWAP_20_n_spread', 'ask_total_volume', 'bid_total_volume', 'five_last_added_volume_a', 'five_last_added_volume_b', 'five_percent_mid_dev_volume_a', 'five_percent_mid_dev_volume_b', 'five_percent_volume_a', 'five_percent_volume_b', 'number_asks', 'number_bids', 'spread', 'ten_last_

In [116]:
temp = BtcUsdDf.astype(float)

In [63]:
A= summarize_order_books_gamma(Database='Cryptoccy', exchange=1, currency_pair=5, n = 5)

FloatProgress(value=0.0, max=5.0)

C:\Users\janpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:127: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [102]:
full_category_list = category_list + statistics_list

In [81]:
len(full_category_list)

53

In [71]:
a = []
b = []

In [82]:
A

,Max_bid,Max_bid_volume,Min_ask,Min_ask_volume,VWAP_10_ask,VWAP_10_bid,VWAP_20_ask,VWAP_20_bid,ask_total_volume,av_ask_volume_price_ratio,...,twenty_percent_volume_a,twenty_percent_volume_b,Mid,spread,VWAP_20_n_bid,VWAP_20_n_ask,VWAP_20_n_spread,VWAP_10_n_bid,log_returns,Vola_last_10
Timestamp,,,,,,,,,,,,,,,,,,,,,
2018-05-14 17:06:01+02:00,731.2200000000,29.2352999200,731.2300000000,20.0773297200,731.2300000000,731.2200000000,731.2300000000,731.2200000000,45160.6973767700,1.160132e+07,...,18062.8547617700,24000.2312773900,731.2250000000,0.00001367568121987076481247222127,-0.000006837840609935382406236110636,0.000006837840609935382406236110636,0.0100000000,-0.000006837840609935382406236110636,NaN,NaN
2018-05-14 17:06:58+02:00,731.7900000000,41.0000000000,731.8000000000,24.6430000000,731.8000000000,731.7900000000,731.8000000000,731.7900000000,43115.0282690400,1.162341e+07,...,16057.7167506600,23557.8563936500,731.7950000000,0.00001366502914067464248867510710,-0.000006832514570337321244337553550,0.000006832514570337321244337553550,0.0100000000,-0.000006832514570337321244337553550,-0.000779,NaN
2018-05-14 17:08:03+02:00,730.9900000000,26.1742132900,731.0000000000,373.6000752900,731.0000000000,730.9900000000,731.0000000000,730.9900000000,44696.2078899000,1.158811e+07,...,17566.1852749000,23486.3943530800,730.9950000000,0.00001367998413121840778664696749,-0.000006839992065609203893323483745,0.000006839992065609203893323483745,0.0100000000,-0.000006839992065609203893323483745,0.001094,NaN
2018-05-14 17:08:59+02:00,730.1800000000,0.6846407600,730.3900000000,3.6647621400,730.39633523786000000000,730.17068464076000000000,730.40485646358000000000,730.16618464076000000000,44498.6405641400,1.157493e+07,...,17212.0410217200,23240.9609498200,730.2850000000,0.0002875589667047796408251573016,-0.0001626972472938647240460915944,0.0001641228610473993030118378441,0.23867182282000000000,-0.0001565352694359051603141239379,0.000972,NaN
2018-05-14 17:09:59+02:00,728.7200000000,15.3155967900,728.7600000000,11.0000000000,728.7600000000,728.7200000000,728.78700000000000000000,728.71765779839500000000,44628.3611294500,1.161900e+07,...,17566.7905391500,24678.9953263300,728.7400000000,0.00005488926091610176468973845267,-0.00003065867333342481543486016961,0.00006449488157641957351044268189,0.06934220160500000000,-0.00002744463045805088234486922634,0.002118,NaN
2018-05-14 17:10:59+02:00,727.1700000000,2.2116529700,727.1900000000,23.0000000000,727.1900000000,727.15442330594000000000,727.1900000000,727.15221165297000000000,44583.0253149900,1.158814e+07,...,17330.2416999900,24404.1027476500,727.1800000000,0.00002750350669710388074479496136,-0.00003821384943205258670480486262,0.00001375175334855194037239748068,0.03778834703000000000,-0.00003517243881844935229241728320,0.002143,NaN
2018-05-14 17:12:03+02:00,727.9800000000,31.1000000000,727.9900000000,36.8084066000,727.9900000000,727.9800000000,727.9900000000,727.9800000000,44286.3958631500,1.159694e+07,...,17178.4946493600,23904.6688688600,727.9850000000,0.00001373654676950761348104699959,-0.000006868273384753806740523499797,0.000006868273384753806740523499797,0.0100000000,-0.000006868273384753806740523499797,-0.001106,NaN
2018-05-14 17:13:03+02:00,725.8300000000,18.4237135600,725.8400000000,63.0875501800,725.8400000000,725.8300000000,725.8400000000,725.82921185678000000000,44215.2127448700,1.162788e+07,...,17183.7038112500,22139.2014052400,725.8350000000,0.00001377723587316676655162674713,-0.000007974461440961099974512113635,0.000006888617936583383275813373563,0.01078814322000000000,-0.000006888617936583383275813373563,0.002958,NaN
2018-05-14 17:14:03+02:00,726.4200000000,24.6883023000,726.4300000000,19.0356000000,726.4300000000,726.4200000000,726.43048220000000000000,726.4200000000,44235.2111844200,1.161460e+07,...,17179.9431788800,23189.7928575900,726.4250000000,0.00001376604604742402863337577864,-0.000006883023023712014316687889321,0.000007546821764118800977389269367,0.01048220000000000000,-0.00000688302302371

In [61]:
A.shape[1]

50

In [78]:
a

[53]

In [79]:
b

[50]

In [104]:
test = get_full_information(currency_pair='BTC/USD', exchange='BitFinex')

FloatProgress(value=0.0, max=3.0)

1


FloatProgress(value=0.0, max=1.0)

C:\Users\janpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:126: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\janpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



FloatProgress(value=0.0, max=1.0)

...

FloatProgress(value=0.0, max=1.0)

FloatProgress(value=0.0, max=1.0)

3


FloatProgress(value=0.0, max=1.0)

FloatProgress(value=0.0, max=1.0)

In [105]:
test

,Max_bid,Max_bid_volume,Mid,Min_ask,Min_ask_volume,Timestamp,VWAP_10_ask,VWAP_10_bid,VWAP_10_n_ask,VWAP_10_n_bid,...,top_ten_volume_a,top_ten_volume_b,top_twenty_volume_a,top_twenty_volume_b,twenty_last_added_volume_a,twenty_last_added_volume_b,twenty_percent_mid_dev_volume_a,twenty_percent_mid_dev_volume_b,twenty_percent_volume_a,twenty_percent_volume_b
2018-05-14 17:06:02+02:00,8760.3000000000,0.5800000000,8760.3500000000,8760.4000000000,3.7914189600,NaN,8762.23304536400000000000,8757.67088775040000000000,0.0002149509282163383883064032830,-0.0003058225127534858767058393786,...,69.4316260800,8.2117046900,91.5150041400,16.3174559000,81.0091050000,246.6140166800,7237.6651078200,9969.2923422800,3448.3183889900,4356.8934702000
2018-05-14 17:06:58+02:00,8754.4000000000,1.3700000000,8754.6000000000,8754.8000000000,0.7651309200,NaN,8757.20189001520000000000,8752.04473500000000000000,0.0002972026152194275009709181459,-0.0002918768418888355835789185114,...,12.6740891500,48.9438319300,83.7412039000,57.1446833300,637.0731557400,101.6172576300,7216.9322634800,10013.1394625800,3424.2428763200,4378.9161148400
2018-05-14 17:08:03+02:00,8757.0000000000,0.7029999900,8758.4000000000,8759.8000000000,0.0447918300,NaN,8762.64975841620000000000,8751.79136099300000000000,0.0004852208641075995615637559372,-0.0007545486626552795031055900621,...,64.8874345700,9.7546999900,95.3281659900,54.5887679400,595.6373347300,460.7669726000,7221.2750093500,9946.1072778200,3436.7315513800,4318.1902371200
2018-05-14 17:09:02+02:00,8754.9000000000,0.5959954500,8755.2000000000,8755.5000000000,0.0593951200,NaN,8762.65729843380000000000,8751.62900397510000000000,0.0008517564914336622807017543860,-0.0004078714392475328947368421053,...,8.1277089800,48.3110430600,81.6222114200,65.9182847500,101.1349880000,766.6669209800,7230.6142185200,9941.3293134100,3444.7420615100,4319.7841679200
2018-05-14 17:09:59+02:00,8750.0000000000,29.1650100000,8750.0500000000,8750.1000000000,4.9931720400,NaN,8750.60028279600000000000,8750.0000000000,0.00006288910303369695030314112491,-0.000005714253061411077650984851515,...,22.0996171000,39.4155565000,112.8870088400,50.8038565000,415.3395870700,18448.1821402700,7037.8469015300,9911.7651465200,3253.2609329700,4085.3537066700
2018-05-14 17:10:58+02:00,8736.7000000000,2.1600000000,8736.7500000000,8736.8000000000,5.9933215800,NaN,8737.40090176300000000000,8734.29388065390000000000,0.00007450158960711935216184507969,-0.0002811250574984977251266203108,...,29.8758359700,10.4360297300,49.2581221800,21.8915768700,316.0159098400,316.9462806100,6812.9836378500,9928.4246261300,3174.6897901800,4097.2930289100
2018-05-14 17:12:03+02:00,8742.5000000000,1.1928000000,8742.5500000000,8742.6000000000,0.2696795100,NaN,8750.18280508170000000000,8742.29941600000000000000,0.0008730639323423943815019645298,-0.00002866257556433763604440352071,...,5.0751011800,17.5203081200,15.5252414100,32.9964529500,120.5434093300,179.3053218200,6810.1756734000,9934.6558247100,3170.8995482900,4101.4914449100
2018-05-14 17:13:02+02:00,8743.5000000000,14.2224673600,8743.5500000000,8743.6000000000,14.2828022200,NaN,8743.6000000000,8743.5000000000,0.000005718501066500448902333720285,-0.000005718501066500448902333720285,...,32.7870579600,24.2804673600,49.8352937900,28.3037890900,93.1035596600,35.5327137800,6828.6863184700,9901.0452511400,3268.4734705500,4062.8448269400
2018-05-14 17:14:01+02:00,8749.4000000000,0.0804220800,8749.4500000000,8749.5000000000,0.1255746300,NaN,8755.01837062010000000000,8748.98563847640000000000,0.0006364252175965346393201858403,-0.00005307322444267925412454497140,...,5.2924658500,22.0860484500,93.2531400100,34.1049263900,113.8971529600,516.1630200800,6788.8106737900,9934.5324373200,3227.6695606500,4096.5241028700
2018-05-14 17:15:03+02:00,8749.1000000000,2.7388986200,8749.1500000000,8749.2000000000,14.6775179200,NaN,8749.2000000000,8746.51791621350000000000,0.000005714840870255967722578764794,-0.0003008387999405656549493379357,...,23.1345274400,11.7413558100,43.

In [22]:
def adapt_name(x,appendix):
    #result = x + '_' + appendix
    return(x + '_' + appendix)

def information_with_all_restrictions(currency_pair, exchange):
    restrictions = [2,3,4,5]
    amounts = [100,#100 top entries\
               25, #25top percent\
               50, #50 last entered entries\
               50]  #50 percecnt deviation from mid price
    df0 = get_full_information(currency_pair, exchange, restrictions[0], amounts[0] )
    df0.rename(lambda x: adapt_name(x,'R2'), axis = 'columns', inplace = True)
    df0.reset_index(inplace = True)
    
    df1 = get_full_information(currency_pair, exchange, restrictions[1], amounts[1] )
    df1.rename(lambda x: adapt_name(x,'R3'), axis = 'columns', inplace = True)
    df1.reset_index(inplace = True)
    
    df2 = get_full_information(currency_pair, exchange, restrictions[2], amounts[2] )
    df2.rename(lambda x: adapt_name(x,'R4'), axis = 'columns', inplace = True)
    df2.reset_index(inplace = True)
    
    df3 = get_full_information(currency_pair, exchange, restrictions[3], amounts[3] )
    df3.rename(lambda x: adapt_name(x,'R5'), axis = 'columns', inplace = True)
    df3.reset_index(inplace = True)
    
    results = pd.concat([df0, df1, df2, df3], axis = 1).set_index('index')
    results.rename(lambda x: x[0], axis = 'index', inplace = True)
    return(results)

In [92]:
BTC_USD_BitFinex_data = information_with_all_restrictions('BTC/USD', 'BitFinex')

FloatProgress(value=0.0, max=6.0)

1


FloatProgress(value=0.0, max=4.0)

C:\Users\janpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



FloatProgress(value=0.0, max=4.0)

...

FloatProgress(value=0.0, max=4.0)

FloatProgress(value=0.0, max=4.0)

FloatProgress(value=0.0, max=4.0)

6


FloatProgress(value=0.0, max=4.0)

In [311]:
Results_BTC_Bitfinex.to_hdf('C:/Users/janpa/Desktop/Uni/Bachelorarbeit/gathered_stats/BTC_USD_all_dbs_BitFinex.h5', \
               key = 'BitFinex', mode = 'w')

C:\Users\janpa\Anaconda3\lib\site-packages\pandas\io\pytables.py:280: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->axis1] [items->None]

  f(store)
C:\Users\janpa\Anaconda3\lib\site-packages\pandas\core\generic.py:1299: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->['Max_bid', 'Max_bid_volume', 'Mid', 'Min_ask', 'Min_ask_volume', 'VWAP_20_ask', 'VWAP_20_bid', 'VWAP_n_ask', 'VWAP_n_bid', 'ask_total_volume', 'bid_total_volume', 'number_asks', 'number_bids', 'spread']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


Av_volume normalized to (price-mid)/volume
feature VOLA_10 implemented
